In [1]:
import csv
import re
import numpy as np
import nltk
import pickle
from categories import categories
%matplotlib inline

from nltk import word_tokenize

In [2]:
qfile = open('question_train.csv', 'r')
qreader = csv.reader(qfile)

qfile.seek(0);
#cols = list(zip(np.arange(21),next(reader)))
j = 0
for i, q in zip(np.arange(21), next(qreader)):
    if j == 3: l = "\n"; j = 0;
    else: l = ""; j += 1
        
    print('{0:2}: {1:25}'.format(i,q), end=l)

 0: question_id               1: user_id                   2: sms_guru_id               3: category_main_id         
 4: question                  5: description               6: tags                      7: categories               
 8: url                       9: rating_count_positive    10: rating_count_negative    11: answer_count             
12: reported                 13: answered                 14: active                   15: deleted                  
16: seo_locked               17: editor_locked            18: editor_id                19: created_at               
20: updated_at               

# Reading out questions and tokenizing, checking vocabulary

In [3]:
cats = categories()

qcatfile = open('question_category_train.csv', 'r')
qcatreader = csv.reader(qcatfile)

next(qcatreader) # skipping column discription

qcat_dict = {} # mapping from question_id to the parent category_id
catq_count = nltk.FreqDist() # maps the category_id to its

for qcat in qcatreader:
    cat_id = int(qcat[1])
    pcat_id = cats.parent_id(cat_id)
    q_id = int(qcat[2])
    
    qcat_dict[q_id] = pcat_id
    catq_count[cats.name(cat_id)] += 1
    
#catq_count.plot()

In [4]:
qfile = open('question_train.csv', 'r')
qreader = csv.reader(qfile)

qfile.seek(0);
next(qreader)

questions = []
vocabulary = {}
for cat_name in cats.all_names():
    vocabulary[cat_name] = nltk.FreqDist()

# Set this parameter to TRUE if you want to read through
# all questions again, elsewise from file set to FALSE.
NewRead = True 

if NewRead:
    for row in qreader:
        if len(row) == 21:
            if int(row[0]) in qcat_dict.keys():
                words = word_tokenize(row[4].lower())
                cat_id = qcat_dict[int(row[0])]
                questions += [{"words": words, "cat_id": cat_id}]
                vocabulary[ cats.name(cat_id) ] += nltk.FreqDist(words)
        
    ## Saving into pickle files
    q_file, v_file = open('questions.pkl', 'wb'), open('vocabulary.pkl', 'wb')
    pickle.dump(questions, q_file)
    pickle.dump(vocabulary, v_file)
    
else:
    ## Loading from pickle files
    q_file, v_file = open('questions.pkl', 'rb'), open('vocabulary.pkl', 'rb')
    questions, vocabulary = pickle.load(q_file), pickle.load(v_file)

In [11]:
print(cats.all_names())
vocabulary['film_and_musik'].most_common(50)

['film_and_musik', 'stars_and_promis', 'computer_and_pc', 'alltag', 'namensforschung', 'literatur_and_sprache', 'schule', 'mensch_and_koerper', 'freizeit_and_sport', 'wissen', 'liebe_and_beziehung', 'astrologie', 'games_and_spiele', 'adult']


[('?', 632),
 ('die', 270),
 ('der', 246),
 ('wie', 239),
 ('von', 231),
 ('ist', 164),
 ('in', 159),
 ('wann', 144),
 ('das', 143),
 ('und', 133),
 ("''", 129),
 ('wer', 97),
 ('was', 94),
 ('es', 89),
 ('kommt', 80),
 ('heißt', 75),
 ('film', 74),
 ('lied', 70),
 (',', 65),
 ('den', 61),
 ('bei', 59),
 ('auf', 59),
 ('hat', 53),
 ('im', 52),
 ('.', 50),
 ('ein', 48),
 ('gibt', 48),
 ('wo', 46),
 ('dem', 45),
 ('sind', 43),
 ('ich', 40),
 ('mit', 40),
 ('aus', 38),
 ('alt', 35),
 ('wird', 35),
 ('eine', 34),
 ('album', 31),
 ('band', 30),
 ('serie', 27),
 ('für', 27),
 ('spielt', 26),
 ('neue', 26),
 ('welche', 25),
 ('oder', 25),
 ('the', 25),
 ('nächste', 24),
 ('staffel', 24),
 ('!', 22),
 ('warum', 22),
 ('raus', 22)]

In [5]:
example = {}
example['words'] = word_tokenize("Geile ficken Titten!".lower())
example

{'words': ['geile', 'ficken', 'titten', '!']}

In [33]:
vocab = nltk.FreqDist()
for cat_name in cats.all_names():
    vocab += nltk.FreqDist(vocabulary[cat_name].most_common(100))
vocab = list(vocab)
feature_set = [(simple_features(vocab, q), cats.name(q['cat_id'])) for q in questions[:4500]]
train_set, test_set = feature_set[:4000], feature_set[4000:4500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [34]:
q_id = 2110
cats.name(questions[q_id]['cat_id']), classifier.classify(simple_features(vocab, questions[q_id]))

('literatur_and_sprache', 'wissen')

In [50]:
test_freq = nltk.FreqDist(['Hallo', 'mein', 'name', 'ist', 'Paul'])
test_freq2 = nltk.FreqDist(word_tokenize('Hallo mein Name ist Tomas'))
print(*test_freq2.most_common(5))

('mein', 1) ('Name', 1) ('Tomas', 1) ('Hallo', 1) ('ist', 1)


In [35]:
nltk.classify.accuracy(classifier, test_set)

0.16

In [36]:
classifier.classify(simple_features(vocab,example))

NameError: name 'example' is not defined

In [27]:
def simple_features(vocab, question):
    features = {}
    for v in vocab:
        features[v] = v in question['words']
    return features

# Checking Vocabulary

In [22]:
totNum = vocabulary.N()
vocNum = vocabulary.B()
print("""The total number of words is {0}. 
The volume of the vocabulary is {1}.
That makes an percentiage of {2:.2}""".format(totNum,vocNum,vocNum/totNum))

The total number of words is 126685. 
The volume of the vocabulary is 18236.
That makes an percentiage of 0.14


In [75]:
allwordlist = [w for q in questions for w in q[1]]
fd = nltk.FreqDist(allwordlist)

In [102]:
n = 50
delta = 0
rows = 5
epc = int(n/rows)
a = np.arange(n) + delta

for j in range(epc):
    row = [fd.most_common(n)[i] for i in a[j::epc]]
    for v in row:
        print(("{0:>10} x {1:<6}").format(*v),end="")
    print("")

         ? x 11576   bedeutet x 1465       kommt x 947           '' x 649         oder x 506   
       der x 4693         das x 1386          wo x 879       welche x 642         mein x 481   
       wie x 3999         hat x 1351         wer x 810         gibt x 593          bei x 478   
       was x 3919         von x 1260        eine x 781          den x 587         viel x 455   
       ist x 3470         ein x 1135         mit x 769           er x 574         wenn x 452   
       die x 2366          es x 1112        kann x 749          für x 562         welt x 430   
       und x 2255         man x 1075       woher x 728        heißt x 554          sie x 423   
       ich x 2011           , x 1061         auf x 701         sind x 548        haben x 415   
        in x 1717           . x 1004       warum x 687           im x 542        viele x 415   
      name x 1652        wann x 997           am x 683           zu x 530          aus x 380   


In [104]:
vocabArr = sorted(list(vocab))

n = 50
delta = 2000
rows = 5
epc = int(n/rows)
a = np.arange(n) + delta

for j in range(epc):
    row = [vocabArr[i] for i in a[j::epc]]
    print(("- {:20}"*len(row)).format(*row))

- anschlag            - anstecken           - antikatalytische    - antonin             - antwortetst         
- anschliesend        - ansteckend          - antike              - antonio             - anubis              
- anschließen         - anstellen           - antimaterie         - antony              - anunis              
- anschlägt           - antarktis           - antipinoxe          - antreten            - anus                
- anschrift           - anteil              - antisemetismus      - antrieb             - anwalt              
- anschwillt          - antenne             - antisemitismus      - antrittsvorlesung   - anwendungen         
- ansehen             - antennenverhältnis  - antje               - anträge             - anwendungsgebiete   
- ansprechen          - anthony             - anton               - antwort             - anwesen             
- anspruch            - antibabypille       - antonella           - antworten           - anwesend            
-

# Playing area

In [125]:
import re
for q in questions[:10]:
    print([w for w in q[1] if re.search('^W',w)])

['Was']
['WIEVIEL']
['Was']
[]
[]
['Wie']
['Wie']
['Wie']
[]
['Was']


In [21]:
for x in range(1,11):
    print(repr(x).zfill(5), repr(x**2).ljust(5),repr(x**3).ljust(5))

00001 1     1    
00002 4     8    
00003 9     27   
00004 16    64   
00005 25    125  
00006 36    216  
00007 49    343  
00008 64    512  
00009 81    729  
00010 100   1000 


In [33]:
from nltk.corpus import names

In [42]:
type(names.words('male.txt'))

list

# Helps

In [128]:
a = np.arange(100)
[b[a[::3]-i] for i in range(3)]

[array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48,
        51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99]),
 array([99,  2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47,
        50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98]),
 array([98,  1,  4,  7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46,
        49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97])]

In [141]:
a = ['car', 'house', 'dor']
b = ['Cat', 'Dog', 'Mouse']
list(zip(*[a,b]))

[('car', 'Cat'), ('house', 'Dog'), ('dor', 'Mouse')]

# checking for ill-shaped lines

In [6]:
csvfile.seek(0);
i = 0 #count total row number
j = 0 #count false row number

for row in reader:
    i += 1
    if(len(row) != 21):
        #print('line size error in line {} \n'.format(reader.line_num))
        #print('line size is: {} \n'.format(len(row)))
        #print(*row)
        #print('\n')
        j += 1

print("""Total number of rows: {0}
Number of rows with wrong length: {1}:
    fraction: {2:.2f} %""".format(i,j,(j/i)*100))

Total number of rows: 16070
Number of rows with wrong length: 57:
    fraction: 0.35 %
